### ЛЕГЕНДА
Продолжаем пробовать новые роли! На этот раз вы — маркетинговый аналитик в компании, предлагающей услуги клининга.

В набор предоставляемых услуг входят три направления: уборка, химчистка, стирка и глажка.

К вам пришёл руководитель направления «Уборка» и попросил выгрузить статистику по конверсии в найм клинеров.

### ЦЕЛЬ
Определить коэффициент конверсии в найм клинеров.

#### ЗАДАЧИ
Выгрузить данные.
Рассчитать коэффициент конверсии с помощью Pandas.

#### КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)
* Определить параметры запроса к Яндекс.Метрике.
* Выгрузить их по API с помощью Jupyter Notebook.
* Обработать данные в Pandas.
* Посчитать коэффициент конверсии по каналам.

#### ЗНАКОМСТВО С ДАННЫМИ
Для выгрузки отчёта нам необходимы следующие данные:

* источники трафика по модели атрибуции «Последний значимый источник»;
* суммарное количество визитов;
* количество выполнений целевого условия «Найм клинеров» во всех визитах.

In [44]:
# Идентификатор счётчика Яндекс.Метрики сайта: 30177909

# Идентификатор достижения цели «Найм клинеров»: 137595631

In [45]:
import json 
import requests
from pprint import pprint

In [46]:
token = 'AQAAAAAB95q6AAfACLdBhhkWNkBjiL0XU9oCf-o' 
headers = {'Authorization': 'OAuth ' + token}
# Идентификатор счётчика Яндекс.Метрики сайта: 30177909
# Идентификатор достижения цели «Найм клинеров»: 137595631
params = {'metrics': 'ym:s:visits,ym:s:goal137595631reaches',
          'dimensions': 'ym:s:lastsignTrafficSource',
          'date1': '2020-09-01',
          'date2': '2020-09-30',
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
metrika_data = response.json()
# print(metrika_data.keys())
# metrika_data['total_rows']
# pprint(metrika_data['data'][0:5])

In [47]:
import pandas as pd # Импортируем библиотеку pandas под псевдонимом pd

metrika_df = pd.DataFrame(metrika_data['data'])
display(metrika_df.head(10)) # Функция .head(10) позволяет выбрать только 10 первые строк датафрейма 

,dimensions,metrics
0,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[3334.0, 0.0]"
1,"[{'icon_id': '0', 'icon_type': 'traffic-source...","[757.0, 0.0]"
2,"[{'icon_id': '1', 'icon_type': 'traffic-source...","[126.0, 0.0]"
3,"[{'icon_id': '8', 'icon_type': 'traffic-source...","[80.0, 0.0]"
4,"[{'icon_id': '-1', 'icon_type': 'traffic-sourc...","[10.0, 0.0]"
5,"[{'icon_id': '4', 'icon_type': 'traffic-source...","[6.0, 0.0]"
6,"[{'icon_id': '9', 'icon_type': 'traffic-source...","[2.0, 0.0]"


Видим, что нет данных по конверсии, все значения равны 0. Связано с тем, что не собираются данные по цели. Если заглянуть в настройки Яндекс.Метрики, видно, что данные по цели начали собирать с 7 октября.

Меняем период в самом первом блоке кода:

In [48]:
# основа
import json 
import requests
from pprint import pprint
token = 'AQAAAAAB95q6AAfACLdBhhkWNkBjiL0XU9oCf-o' 
headers = {'Authorization': 'OAuth ' + token}

In [49]:
params = {'metrics': 'ym:s:visits,ym:s:goal137595631reaches',
          'dimensions': 'ym:s:lastsignTrafficSource', # атрибуция по последнему значимому источнику. 
          #'dimensions': 'ym:s:lastTrafficSource', # атрибуция по последнему источнику; для задания 47.6.2
          'date1': '2020-10-07',  
          'date2': '2020-10-09',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
metrika_data = response.json()

Избавляемся от вложенных словарей.
Преобразуем данные в так называемый плоский вид, то есть избавимся от вложенных словарей.

In [50]:
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

Обратите внимание на использование функции enumerate(). Она позволяет сделать цикл по списку, при этом возвращая сам элемент списка, а также его индекс в этом списке. Это может быть удобно, если нужно сделать вывод информационного сообщения о количестве пройденных итераций цикла. Также эта функция бывает полезна, если нужно параллельно обращаться к соответствующему элементу другого списка, как в случае нашей функции getMetrikaDataInListOfDicts().

In [51]:
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:5])

[{'ym:s:goal137595631reaches': 10.0,
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:visits': 345.0},
 {'ym:s:goal137595631reaches': 5.0,
  'ym:s:lastsignTrafficSource': 'Direct traffic',
  'ym:s:visits': 113.0},
 {'ym:s:goal137595631reaches': 1.0,
  'ym:s:lastsignTrafficSource': 'Link traffic',
  'ym:s:visits': 16.0},
 {'ym:s:goal137595631reaches': 1.0,
  'ym:s:lastsignTrafficSource': 'Social network traffic',
  'ym:s:visits': 12.0},
 {'ym:s:goal137595631reaches': 0.0,
  'ym:s:lastsignTrafficSource': 'Internal traffic',
  'ym:s:visits': 2.0}]


Создаём датафрейм

In [52]:
# Передадим metrika_list_of_dicts в функцию pd.DataFrame(), чтобы создать датафрейм metrika_df:

metrika_df = pd.DataFrame(metrika_list_of_dicts)
display(metrika_df.head(10))

,ym:s:lastsignTrafficSource,ym:s:visits,ym:s:goal137595631reaches
0,Search engine traffic,345.0,10.0
1,Direct traffic,113.0,5.0
2,Link traffic,16.0,1.0
3,Social network traffic,12.0,1.0
4,Internal traffic,2.0,0.0


In [53]:
# Для удобства переименуем столбцы и сразу же добавим новый столбец с расчётом.

metrika_df.columns=['Source','Visits','Conversions']
metrika_df['Last non-direct CR%'] = metrika_df['Conversions']/metrika_df['Visits']
display(metrika_df.head(10))

,Source,Visits,Conversions,Last non-direct CR%
0,Search engine traffic,345.0,10.0,0.028986
1,Direct traffic,113.0,5.0,0.044248
2,Link traffic,16.0,1.0,0.062500
3,Social network traffic,12.0,1.0,0.083333
4,Internal traffic,2.0,0.0,0.000000


In [60]:
params2 = {'metrics': 'ym:s:visits,ym:s:goal137595631reaches',
          'dimensions': 'ym:s:lastTrafficSource', # атрибуция по последнему источнику; для задания 47.6.2
          'date1': '2020-10-07',  
          'date2': '2020-10-09',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
response2 = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params2, headers=headers)
metrika_data_2 = response2.json()

def getMetrikaDataInListOfDicts(metrika_data_2):
    list_of_dicts = []
    dimensions_list = metrika_data_2['query']['dimensions']
    metrics_list = metrika_data_2['query']['metrics']
    for data_item in metrika_data_2['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data_2)
metrika_df2 = pd.DataFrame(metrika_list_of_dicts)
metrika_df2.columns=['Source','Visits','Conversions']
metrika_df2['Last click CR%'] = metrika_df2['Conversions']/metrika_df2['Visits']
display(metrika_df2.head(10))

,Source,Visits,Conversions,Last click CR%
0,Search engine traffic,317.0,10.0,0.031546
1,Direct traffic,144.0,6.0,0.041667
2,Link traffic,15.0,1.0,0.066667
3,Social network traffic,10.0,0.0,0.000000
4,Internal traffic,2.0,0.0,0.000000


In [61]:
# Задание 47.6.4 К какому каналу была отнесена конверсия из источника Direct в модели Last non-direct?
df1 = metrika_df.loc[:,['Source','Last non-direct CR%']]
df2 = metrika_df2.loc[:,['Source','Last click CR%']]
attribution_compare_df = df1.merge(df2, on='Source')
attribution_compare_df['Last click to Last non-direct Difference'] = attribution_compare_df['Last click CR%']/attribution_compare_df['Last non-direct CR%'] -1
attribution_compare_df

,Source,Last non-direct CR%,Last click CR%,Last click to Last non-direct Difference
0,Search engine traffic,0.028986,0.031546,0.088328
1,Direct traffic,0.044248,0.041667,-0.058333
2,Link traffic,0.062500,0.066667,0.066667
3,Social network traffic,0.083333,0.000000,-1.000000
4,Internal traffic,0.000000,0.000000,NaN


ответ 47.6.4 Social network traffic , т.к. разница - 1